Deadline: 23:59 on 2021 October 18

# TODO

* Prepare data
    * Images to tensors
    * Bounding box infomration to numpy array
* Implement two different models
    * Representation of the images as training input
    * Representation of the bounding boxes as objective
        * HINT: the binary classification of pixels as belonging to a bounding box or not
* Test and evaluate
    * Way 1: choose a probability threshold to decide whether a pixel is inside the bounding box or not, and then take recall/precision/X11/accuracy
    * Way 2: report it in terms of error, such as mean squared error


In [19]:
# Maybe not needed
'''
with open(info_filename, 'r') as info_file:
    data = info_file.read()
info_obj = json.loads(data)

df_train = pd.DataFrame.from_dict(info_obj['train'])
df_val = pd.DataFrame.from_dict(info_obj['val'])
df = pd.concat([df_train, df_val])
df = df[pd.DataFrame(df.file_name.to_list()).isin(image_files).any(1).values]
'''

"\nwith open(info_filename, 'r') as info_file:\n    data = info_file.read()\ninfo_obj = json.loads(data)\n\ndf_train = pd.DataFrame.from_dict(info_obj['train'])\ndf_val = pd.DataFrame.from_dict(info_obj['val'])\ndf = pd.concat([df_train, df_val])\ndf = df[pd.DataFrame(df.file_name.to_list()).isin(image_files).any(1).values]\n"

In [20]:
import json
import jsonlines
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from torchvision import transforms
import matplotlib.pyplot as plt
import matplotlib.path as mplpath
import matplotlib.image as mpimg

convert_tensor = transforms.ToTensor()

source_dir = "/scratch/lt2326-h21/a1/"
image_dir = "/scratch/lt2326-h21/a1/images/"
info_filename = "/scratch/lt2326-h21/a1/info.json"

%matplotlib inline

In [21]:
image_files = [f for f in listdir(image_dir) if isfile(join(image_dir, f))]
image_files = image_files[:100]

In [22]:
def get_image_data_from_jsonl(jsonl_file, included_images):
   image_data = []
   images = []
   with jsonlines.open(jsonl_file) as reader:
      for line in reader.iter(type=dict):
         if (line.get('file_name') in included_images):
            image_data.append(line)
            image = mpimg.imread(image_dir + line.get('file_name'))
            images.append(image)
            
   return image_data, images

In [23]:
# Setup coordinates grid to use for getting polygon matrix
height = 2048
width = 2048
rows = np.arange(height)
cols = np.arange(width)
coordinates_grid = np.empty((height, width, 2), dtype=np.intp)
coordinates_grid[..., 0] = rows[:, None]
coordinates_grid[..., 1] = cols
coordinates_grid.shape = (height*width, 2)

In [24]:
#training_data = [] # lista av tuples, med bild och ground truth
def get_image_polygon_data(image_data):
    image_polygon_data = []

    for entry in image_data:
        image_dict = { 
            'file_name': entry['file_name'],
            'polygons': []
            }
        for annotation in entry['annotations']:
            for bbox in annotation:
                if(bbox['is_chinese']):
                    image_dict['polygons'].append(bbox['polygon'])
        image_polygon_data.append(image_dict)
    return image_polygon_data


In [25]:
def create_training_data(polygon_data):
    training_data = []

    for image in polygon_data:
        img = mpimg.imread(image_dir + image['file_name'])
        img_tensor = convert_tensor(img)
        #chinese_chars = np.empty((height, width), dtype=np.intp)
        chinese_chars = np.empty((height*width))
        for polygon in image['polygons']:
            path = mplpath.Path(polygon)
            polygon_grid = np.asarray(path.contains_points(coordinates_grid), int)
            chinese_chars += polygon_grid
            
        chinese_chars.shape = (height, width)
        training_data.append((img_tensor, chinese_chars))
    return training_data


In [26]:
def get_ground_truth(image_data):
    truth_grid = np.zeros((height, width), dtype=np.intp)
    for polygon in image_data['polygons']:
        path = mplpath.Path(polygon)
        polygon_grid = np.asarray(path.contains_points(coordinates_grid), int)
        polygon_grid.shape = (height, width)
        truth_grid += polygon_grid
    
    truth_grid = np.resize(truth_grid, (200,200))
    return truth_grid

In [27]:
image_training_data, images = get_image_data_from_jsonl(source_dir + "train.jsonl", image_files)
image_val_data, _ = get_image_data_from_jsonl(source_dir + "val.jsonl", image_files)
print(len(images))

86


In [28]:
training_data_polygons = get_image_polygon_data(image_training_data)
#get_ground_truth(training_data_polygons[0])
#training_data = create_training_data(training_data_polygons)

In [29]:
import torch
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as F

In [30]:
# Credit: https://amaarora.github.io/2020/09/13/unet.html

# Input - bilder / tre dimensionell array
# Output - Koordinater till bounding box
# Dimension på bilderna: 2048 x 2048 x 3
# Dimension på bounding box: 2 x 4
# Dimension på output: 2048 x 2048 x 1 (binär data över var kinesiska tecken var)

# Två saker!
#   1. Hitta tecken
#   2. Avgör om det är kinesiskt eller inte

class charDetectionCNN(nn.Module):

# RuntimeError: Expected 4-dimensional input for 4-dimensional weight [3, 3, 3, 3], 
# but got 3-dimensional input of size [3, 2048, 2048] instead
    def __init__(self):
        super().__init__()
        chan_0 = 3
        chan_1 = 16
        chan_2 = 32
        self.out_sz = (200, 200)

        # Encode Block 1
        self.conv_encode_11 = nn.Conv2d(chan_0, chan_1, 3)
        self.relu_encode_1  = nn.ReLU()
        self.conv_encode_12 = nn.Conv2d(chan_1, chan_1, 3)
        # Encode Block 2
        self.conv_encode_21 = nn.Conv2d(chan_1, chan_2, 3)
        self.relu_encode_2  = nn.ReLU()
        self.conv_encode_22 = nn.Conv2d(chan_2, chan_2, 3)

        self.pool = nn.MaxPool2d(2)

        # Decoded Block 1
        self.upconv1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv_decode_11 = nn.Conv2d(chan_2, chan_1, 3)
        self.relu_decode_1  = nn.ReLU()
        self.conv_decode_12 = nn.Conv2d(chan_1, chan_1, 3)

        # Decoded Block 2
        self.upconv2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv_decode_21 = nn.Conv2d(chan_1, chan_0, 3)
        self.relu_decode_2  = nn.ReLU()
        self.conv_decode_22 = nn.Conv2d(chan_0, chan_0, 3)

        # Final
        self.final = nn.Conv2d(chan_0, 1, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, inp):
        # encode 1

        #print("pre: conv_encode 1:", inp.shape)
        enc1 = self.conv_encode_12(self.relu_encode_1(self.conv_encode_11(inp)))
        
        #print("pre: self.pool(enc1):", enc1.shape)
        enc2 = self.pool(enc1)
        # encode 2
        #print("pre: conv_encode 2:", enc2.shape)
        enc2 = self.conv_encode_22(self.relu_encode_2(self.conv_encode_21(enc2)))
        #print("pre: self.pool(enc2):", enc2.shape)
        #enc2 = self.pool(enc2)
        #print("postpool:", enc2.shape)


        # decode 1
        #print("pre: upconv1(enc2):", enc2.shape)
        x = self.upconv1(enc2)
        #print("post: upconv1(enc2):", x.shape)

        # crop
        #_, _, h, w = x.shape
        #print("In decode 1, x.shape:", x.shape)
        #temp = transforms.CenterCrop([h, w])(enc2)
        #print(temp.shape)
        ## concat
        #x = torch.cat([x, temp], dim=1)
        #print("In decode 1, x.shape:", x.shape)
        x = self.conv_decode_12(self.relu_decode_1(self.conv_decode_11(x)))

        #x = self.conv_decode_11(self.relu_decode_1(self.conv_decode_12(x)))
        # Given groups=1, weight of size [128, 1024, 3, 3], expected input[1, 2048, 188, 188] to have 1024 channels, but got 2048 channels instead
        
        # decode 2
        x = self.upconv2(x)
        #_, _, h, w = x.shape
        #temp = transforms.CenterCrop([h, w])(enc1)
        #x = torch.cat([x, temp], dim=1)
        x = self.conv_decode_22(self.relu_decode_2(self.conv_decode_21(x)))

        out = self.final(x)
        out = self.sigmoid(out)
        out = F.interpolate(out, self.out_sz)
        return out

In [31]:

def get_tensor_image(image_data):
    image = mpimg.imread(image_dir + image_data['file_name'])
    rsize = np.resize(image, (200,200,3))
    return convert_tensor(rsize)

In [32]:
gts = {}
for img in training_data_polygons:
    ground_truth = torch.Tensor(get_ground_truth(img))
    ground_truth = ground_truth.unsqueeze(0)
    ground_truth = ground_truth.unsqueeze(0)
    gts[img['file_name']] = ground_truth

In [33]:
# Orig size = 2048 x 2048 x 3
# Resized = 256 x 256 x 3
# input


# number of neurons in each layer
input_num_units = 256*256
hidden_num_units = 500
output_num_units = 10

# set remaining variables
epochs = 10
learning_rate = 0.0005

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
model = charDetectionCNN()
loss_function = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
model.train()
print(len(training_data_polygons))
for i in range(epochs):
    for image in training_data_polygons:
        tensor_image = get_tensor_image(image)
        output = model(tensor_image.unsqueeze(0))
        loss = loss_function(output, gts[image['file_name']])
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    

86


# TODO imorgon 20211012

1. Gör bilderna mindre (2048 x 2048 -> 200 x 200)
2. Få första modellen att fungera

3. Påbörja projektet
    * Bestäm vad jag ska göra
    * Skaffa data (api?)

x. Försök optimera "ground_truth"-functionen

